In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import librosa
import librosa.display
import tensorflow as tf
import os
import shutil
import pickle
import cv2
import json
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import Optimizer

In [2]:
file_path = 'C:/Users/Akoba/Desktop/START up/Covid19_research_project/data/raw/cough_sounds'
sound_metadata = pd.read_csv(file_path + '/sound-metadata.csv', encoding = 'latin1')

In [3]:
sound_metadata.columns

Index(['USER_ID', 'COUNTRY', 'AGE', 'COVID_STATUS', 'ENGLISH_PROFICIENCY',
       'GENDER', 'COUNTY_RO_STATE', 'CITY_LOCALITY', 'Diabetes', 'Asthma',
       'Smoker', 'Hypertension', 'Fever', 'Returning_User', 'Using_Mask',
       'Cold', 'Caugh', 'Muscle_Pain', 'loss_of_smell', 'Sore_Throat',
       'Fatigue', 'Breathing_Difficulties', 'Chronic_Lung_Disease',
       'Ischemic_Heart_Disease', 'Pneumonia', 'COVID_test_status', 'Diarrheoa',
       'DATES', 'breathing-deep', 'breathing-shallow', 'cough-heavy',
       'cough-shallow', 'counting-fast', 'counting-normal', 'vowel-a',
       'vowel-e', 'vowel-o'],
      dtype='object')

In [4]:
sound_metadata.drop('COUNTRY', axis=1, inplace=True)
sound_metadata.drop('GENDER', axis=1, inplace=True)
sound_metadata.drop('COUNTY_RO_STATE', axis=1, inplace=True)
sound_metadata.drop('CITY_LOCALITY', axis=1, inplace=True)
sound_metadata.drop('ENGLISH_PROFICIENCY', axis=1, inplace=True)
sound_metadata.drop('DATES', axis=1, inplace=True)

In [5]:
sound_metadata.columns

Index(['USER_ID', 'AGE', 'COVID_STATUS', 'Diabetes', 'Asthma', 'Smoker',
       'Hypertension', 'Fever', 'Returning_User', 'Using_Mask', 'Cold',
       'Caugh', 'Muscle_Pain', 'loss_of_smell', 'Sore_Throat', 'Fatigue',
       'Breathing_Difficulties', 'Chronic_Lung_Disease',
       'Ischemic_Heart_Disease', 'Pneumonia', 'COVID_test_status', 'Diarrheoa',
       'breathing-deep', 'breathing-shallow', 'cough-heavy', 'cough-shallow',
       'counting-fast', 'counting-normal', 'vowel-a', 'vowel-e', 'vowel-o'],
      dtype='object')

In [8]:
print(sound_metadata['COVID_STATUS'].unique)

<bound method Series.unique of 0                       healthy
1                       healthy
2                       healthy
3                       healthy
4                       healthy
                 ...           
1392    no_resp_illness_exposed
1393                    healthy
1394              positive_mild
1395                    healthy
1396                    healthy
Name: COVID_STATUS, Length: 1397, dtype: object>


In [11]:
uni_val = sound_metadata['COVID_STATUS'].nunique()
print("Number of unique values in 'column_name':", uni_val)

Number of unique values in 'column_name': 8


In [12]:
def label_for_cat(file_path):
    if 'train' in file_path:
        res = 'train'
    else:
        res = 'test'
    return res

In [13]:
sound_metadata['COVID_test_status'] = sound_metadata['COVID_test_status'].astype('str')
sound_metadata['COVID_LABEL'] = np.vectorize(label_for_cat)(sound_metadata['COVID_test_status'])

In [14]:
sound_metadata.columns
sound_metadata.count()

USER_ID                   1397
AGE                       1397
COVID_STATUS              1397
Diabetes                  1397
Asthma                    1397
Smoker                    1397
Hypertension              1397
Fever                     1397
Returning_User            1397
Using_Mask                1397
Cold                      1397
Caugh                     1397
Muscle_Pain               1397
loss_of_smell             1397
Sore_Throat               1397
Fatigue                   1397
Breathing_Difficulties    1397
Chronic_Lung_Disease      1397
Ischemic_Heart_Disease    1397
Pneumonia                 1397
COVID_test_status         1397
Diarrheoa                 1397
breathing-deep            1396
breathing-shallow         1396
cough-heavy               1396
cough-shallow             1395
counting-fast             1397
counting-normal           1397
vowel-a                   1396
vowel-e                   1396
vowel-o                   1395
COVID_LABEL               1397
dtype: i

In [15]:
sound_metadata.groupby(['COVID_LABEL','COVID_test_status']).agg({'USER_ID':'count'}).rename(columns={'USER_ID':'User Count'})

User Count
COVID_LABEL COVID_test_status            
test        0.0                      1341
            1.0                        14
            nan                        42

In [16]:
# Fill NaN values in 'COVID_test_status' column with 1.0
sound_metadata['COVID_test_status'].fillna(1.0, inplace=True)

C:\Users\Akoba\AppData\Local\Temp\ipykernel_21548\1047930978.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sound_metadata['COVID_test_status'].fillna(1.0, inplace=True)


In [17]:
sound_metadata.groupby(['COVID_LABEL','COVID_test_status']).agg({'USER_ID':'count'}).rename(columns={'USER_ID':'User Count'})

User Count
COVID_LABEL COVID_test_status            
test        0.0                      1341
            1.0                        14
            nan                        42

In [18]:
sound_metadata.count()

USER_ID                   1397
AGE                       1397
COVID_STATUS              1397
Diabetes                  1397
Asthma                    1397
Smoker                    1397
Hypertension              1397
Fever                     1397
Returning_User            1397
Using_Mask                1397
Cold                      1397
Caugh                     1397
Muscle_Pain               1397
loss_of_smell             1397
Sore_Throat               1397
Fatigue                   1397
Breathing_Difficulties    1397
Chronic_Lung_Disease      1397
Ischemic_Heart_Disease    1397
Pneumonia                 1397
COVID_test_status         1397
Diarrheoa                 1397
breathing-deep            1396
breathing-shallow         1396
cough-heavy               1396
cough-shallow             1395
counting-fast             1397
counting-normal           1397
vowel-a                   1396
vowel-e                   1396
vowel-o                   1395
COVID_LABEL               1397
dtype: i

In [19]:
sound_metadata.columns

Index(['USER_ID', 'AGE', 'COVID_STATUS', 'Diabetes', 'Asthma', 'Smoker',
       'Hypertension', 'Fever', 'Returning_User', 'Using_Mask', 'Cold',
       'Caugh', 'Muscle_Pain', 'loss_of_smell', 'Sore_Throat', 'Fatigue',
       'Breathing_Difficulties', 'Chronic_Lung_Disease',
       'Ischemic_Heart_Disease', 'Pneumonia', 'COVID_test_status', 'Diarrheoa',
       'breathing-deep', 'breathing-shallow', 'cough-heavy', 'cough-shallow',
       'counting-fast', 'counting-normal', 'vowel-a', 'vowel-e', 'vowel-o',
       'COVID_LABEL'],
      dtype='object')

In [20]:
# Correcting COVID_test_status based on COVID_STATUS
sound_metadata.loc[sound_metadata['COVID_STATUS'].astype(str).str.contains('positive'), 'COVID_test_status'] = 1.0

# Verify the changes (Optional but recommended)
sound_metadata.groupby(['COVID_LABEL','COVID_test_status','COVID_STATUS']).agg({'USER_ID':'count'}).rename(columns={'USER_ID':'User Count'})

User Count
COVID_LABEL COVID_test_status COVID_STATUS                           
test        1.0               positive_asymp                       11
                              positive_mild                        38
                              positive_moderate                     7
            0.0               healthy                            1127
                              no_resp_illness_exposed              84
                              recovered_full                       10
                              resp_illness_not_identified          77
                              ñ                                     1
            nan               healthy                              35
                              no_resp_illness_exposed               6
                              resp_illness_not_identified           1

In [ ]:
print(sound_metadata.isnull().sum())

In [ ]:
# Fill missing values with a placeholder (e.g., empty string or a default path)
sound_metadata.fillna('', inplace=True)

In [ ]:
source = sound_metadata['COVID_test_status'].value_counts()
print(source.count)

In [ ]:
import nbformat
print(nbformat.__version__)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings("ignore")

fig = go.Figure(data=[go.Pie(labels=source.index,values=source.values)])
fig.update_layout(title='Target distribution')
fig.show()

In [ ]:
print(sound_metadata['COVID_test_status'])

In [ ]:
for col in ['breathing-deep', 'breathing-shallow', 'cough-heavy', 'cough-shallow',
            'counting-fast', 'counting-normal', 'vowel-a', 'vowel-e', 'vowel-o']:
    sound_metadata[col] = sound_metadata[col].apply(lambda x: os.path.join(file_path, x.lstrip('/')))

In [ ]:
print(sound_metadata.head())

In [ ]:
# Function to extract MFCC features from an audio file
def extract_mfcc(file_path, n_mfcc=13):
    if not file_path:  # Handle empty paths
        return np.zeros((n_mfcc,))
    try:
        y, sr = librosa.load(file_path, sr=None)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        mfccs = np.mean(mfccs.T, axis=0)
        return mfccs
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return np.zeros((n_mfcc,))

In [ ]:
# Extract MFCC features for all audio files in the metadata
for col in ['breathing-deep', 'breathing-shallow', 'cough-heavy', 'cough-shallow',
            'counting-fast', 'counting-normal', 'vowel-a', 'vowel-e', 'vowel-o']:
    sound_metadata[col + '_mfcc'] = sound_metadata[col].apply(lambda x: extract_mfcc(x))

# Combine all MFCC features into a single feature set
mfcc_features = np.hstack([sound_metadata[col + '_mfcc'].tolist() for col in ['breathing-deep', 'breathing-shallow', 'cough-heavy', 'cough-shallow',
                 'counting-fast', 'counting-normal', 'vowel-a', 'vowel-e', 'vowel-o']])

In [ ]:
# Separate features and labels
X = mfcc_features
y = sound_metadata['COVID_LABEL']
y = to_categorical(y) 

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Reshape the data to fit the RNN model
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
class Coati(tf.keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.01, name='Coati', **kwargs):
        """Constructs a new Coati optimizer.

        Args:
            learning_rate: A Tensor or a floating point value. The learning rate.
            name: Optional name prefix for the operations created when applying gradients.
            **kwargs: Keyword arguments. Allowed to be one of "clipnorm" or "clipvalue".
        """
        super(Coati, self).__init__(name, **kwargs)
        self._set_hyper('learning_rate', kwargs.get('lr', learning_rate))  # handle lr=learning_rate
        # Add any other hyperparameters you need here

    def _create_slots(self, var_list):
        # Create slots for additional tensors here
        pass

    def _resource_apply_dense(self, grad, var, apply_state=None):
        # Update 'var' with 'grad' here using your custom optimization algorithm
        lr_t = self._decayed_lr(tf.float32)  # handle learning rate decay
        var.assign_sub(lr_t * grad)

    def _resource_apply_sparse(self, grad, var, indices, apply_state=None):
        # Handle sparse gradient updates here if necessary
        lr_t = self._decayed_lr(tf.float32)  # handle learning rate decay
        var.assign_sub(lr_t * tf.gather(grad, indices))

    def get_config(self):
        base_config = super(Coati, self).get_config()
        return {**base_config, 'learning_rate': self._serialize_hyperparameter('learning_rate')}

In [ ]:
import types
import torch

def set_default_args():
    args = types.SimpleNamespace()

    # model params
    args.input_size = 12  # == n_mfcc
    args.batch_size = 1
    args.hidden_size = 64
    args.num_layers = 3

    # training params
    args.num_epochs = 100
    args.learning_rate = 0.0001
    args.learning_rate_decay_interval = 5 # decay for every 5 epochs
    args.learning_rate_decay_rate = 0.5 # lr = lr * rate
    args.weight_decay = 0.00
    args.gradient_accumulations = 16 # number of gradient accums before step
    
    # training params2
    args.load_weights_from = None
    args.finetune_model = False # If true, fix all parameters except the fc layer
    args.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # data
    args.data_folder = "data/data_train/"
    args.train_eval_test_ratio=[0.9, 0.1, 0.0]
    args.do_data_augment = False

    # labels
    args.classes_txt = labels
    args.num_classes = None # should be added with a value somewhere, like this:
    #                = len(lib.read_list(args.classes_txt))

    # log setting
    args.plot_accu = True # if true, plot accuracy for every epoch
    args.show_plotted_accu = False # if false, not calling plt.show(), so drawing figure in background
    args.save_model_to = 'checkpoints/' # Save model and log file
    
    return args


In [ ]:
def load_weights(model, weights, PRINT=False):
    for i, (name, param) in enumerate(weights.items()):
        model_state = model.state_dict()
        
        if name not in model_state:
            print("-"*80)
            print("weights name:", name) 
            print("RNN states names:", model_state.keys()) 
            assert 0, "Wrong weights file"
            
        model_shape = model_state[name].shape
        if model_shape != param.shape:
            print(f"\nWarning: Size of {name} layer is different between model and weights. Not copy parameters.")
            print(f"\tModel shape = {model_shape}, weights' shape = {param.shape}.")
        else:
            model_state[name].copy_(param)


In [ ]:
def create_RNN_model(args, load_weights_from=None):
    args.num_classes = len(labels)
    args.save_log_to = args.save_model_to + "log.txt"
    args.save_fig_to = args.save_model_to + "fig.jpg"
    
    device = args.device
    model = RNN(args.input_size, args.hidden_size, args.num_layers, args.num_classes, device).to(device)
    
    if load_weights_from:
        print(f"Load weights from: {load_weights_from}")
        weights = torch.load(load_weights_from)
        load_weights(model, weights)
    
    return model


In [ ]:
def evaluate_model(model, eval_loader, num_to_eval=-1):
    device = model.device
    correct = 0
    total = 0
    for i, (featuress, labels) in enumerate(eval_loader):

        featuress = featuress.to(device)
        labels = labels.to(device)

        outputs = model(featuress)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if i+1 == num_to_eval:
            break
    eval_accu = correct / total
    print('  Evaluate on eval or test dataset with {} samples: Accuracy = {}%'.format(
        i+1, 100 * eval_accu)) 
    return eval_accu


In [ ]:
def fix_weights_except_fc(model):
    not_fix = "fc"
    for name, param in model.state_dict().items():
        if not_fix in name:
            continue
        else:
            print(f"Fix {name} layer", end='. ')
            param.requires_grad = False
    print("")


In [ ]:
import os
import torch.nn as nn
import matplotlib.pyplot as plt

def train_model(model, args, train_loader, eval_loader):

    device = model.device
    logger = lib.TrainingLog(training_args=args)
    if args.finetune_model:
        fix_weights_except_fc(model)
        
    if args.save_model_to:
        if not os.path.exists(args.save_model_to):
            os.makedirs(args.save_model_to)
            
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)
    optimizer.zero_grad()

    def update_lr(optimizer, lr):    
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

    total_step = len(train_loader)
    curr_lr = args.learning_rate
    cnt_batches = 0
    for epoch in range(1, 1+args.num_epochs):
        cnt_correct, cnt_total = 0, 0
        for i, (featuress, labels) in enumerate(train_loader):
            cnt_batches += 1

            featuress = featuress.to(device)
            labels = labels.to(device)
            
            outputs = model(featuress)
            loss = criterion(outputs, labels)
            
            loss.backward()
            if cnt_batches % args.gradient_accumulations == 0:
                optimizer.step()
                optimizer.zero_grad()

            _, argmax = torch.max(outputs, 1)
            cnt_correct += (labels == argmax.squeeze()).sum().item()
            cnt_total += labels.size(0)
            
            train_accu = cnt_correct/cnt_total
            if (i+1) % 50 == 0 or (i+1) == len(train_loader):
                print ('Epoch [{}/{}], Step [{}/{}], Loss = {:.4f}, Train accuracy = {:.2f}' 
                    .format(epoch, args.num_epochs, i+1, total_step, loss.item(), 100*train_accu))
            continue
        print(f"Epoch {epoch} completes")
        
        if (epoch) % args.learning_rate_decay_interval == 0:
            curr_lr *= args.learning_rate_decay_rate
            update_lr(optimizer, curr_lr)
    
        if (epoch) % 1 == 0 or (epoch) == args.num_epochs:
            eval_accu = evaluate_model(model, eval_loader, num_to_eval=-1)
            if args.save_model_to:
                name_to_save = args.save_model_to + "/" + "{:03d}".format(epoch) + ".ckpt"
                torch.save(model.state_dict(), name_to_save)
                print("Save model to: ", name_to_save)

            logger.store_accuracy(epoch, train=train_accu, eval=eval_accu)
            logger.save_log(args.save_log_to)
            
            if args.plot_accu and epoch == 1:
                plt.figure(figsize=(10, 8))
                plt.ion()
                if args.show_plotted_accu:
                    plt.show()
            if (epoch == args.num_epochs) or (args.plot_accu and epoch>1):
                logger.plot_train_eval_accuracy()
                if args.show_plotted_accu:
                    plt.pause(0.01)
                plt.savefig(fname=args.save_fig_to)
        
        print("-"*80 + "\n")
    
    return


In [ ]:
# Assuming COATI is installed or available in your working directory
from coati import COATI

def train_model(model, args, train_loader, eval_loader):

    device = model.device
    logger = lib.TrainingLog(training_args=args)
    if args.finetune_model:
        fix_weights_except_fc(model)
        
    if args.save_model_to:
        if not os.path.exists(args.save_model_to):
            os.makedirs(args.save_model_to)
            
    criterion = nn.CrossEntropyLoss()
    # Replace Adam with COATI
    optimizer = COATI(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)
    optimizer.zero_grad()

    def update_lr(optimizer, lr):    
        for param_group in optimizer.param_groups:
            param_group


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

In [ ]:
# Print the classification report
print(classification_report(y_true_classes, y_pred_classes))